<a href="https://colab.research.google.com/github/robbybrodie/time_as_computation_cost/blob/main/notebooks/cosmology_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
# TACC Cosmological Expansion and Dark Energy

This experiment explores Friedmann-Lemaître-Robertson-Walker (FLRW) metrics adapted to the TACC framework, investigating how computational capacity constraints can naturally explain dark energy and cosmic acceleration.

## Theory

### Modified Friedmann Equation

- **Standard FLRW**: H² = (8πG/3)ρ describes cosmic expansion
- **TACC Modification**: H² = (8πG/3)(ρ_m + ρ_de × B(a))
- **Computational Capacity**: N ≈ a(t), where a is the scale factor
- **Constitutive Law**: B(N) = exp[-κ(1-N)] modifies dark energy evolution

### Distance-Redshift Relations

- **Luminosity Distance**: d_L = (1+z) ∫[c/H(z')] dz' from 0 to z
- **Distance Modulus**: μ = 5 log₁₀(d_L/10pc) observable in supernovae
- **TACC Predictions**: κ ≠ 2 gives observable deviations from ΛCDM

### Physical Interpretation

The scale factor a(t) represents the universe's computational capacity. Early universe (small a) had limited capacity, while computational constraints naturally drive present acceleration without requiring a cosmological constant.

## Setup: Clone Repository and Install Dependencies

In [ ]:
# Clone and setup (idempotent with forced refresh)
import os, sys, subprocess, shutil, pathlib

REPO_URL = "https://github.com/robbybrodie/time_as_computation_cost.git"
REPO_NAME = "time_as_computation_cost"

# Force fresh clone to avoid caching issues
if pathlib.Path(REPO_NAME).exists():
    !rm -rf $REPO_NAME
    
!git clone $REPO_URL

%cd $REPO_NAME

# Install package
if (pathlib.Path("pyproject.toml")).exists():
    !pip install -e .

# Set random seed for reproducibility
import numpy as np, random
np.random.seed(42)
random.seed(42)

## Run Cosmological Expansion Experiment

In [ ]:
from experiments.run_cosmology import main
main()

## Display Results

In [ ]:
from IPython.display import Image, display
from pathlib import Path

# Display generated plots
output_dir = Path("experiments/out/cosmology")

print("TACC Cosmological Expansion Analysis:")
display(Image(str(output_dir / "cosmological_expansion.png")))

print("\nSupernova Data Fitting:")
display(Image(str(output_dir / "supernova_fit.png")))

print("\nParameter Constraints:")
display(Image(str(output_dir / "parameter_constraints.png")))

print("\nDark Energy Evolution:")
display(Image(str(output_dir / "dark_energy_evolution.png")))

# Display numerical results
print("\nNumerical Results:")
with open(output_dir / "results.txt", 'r') as f:
    print(f.read())

## Interactive Parameter Exploration

In [ ]:
# Interactive exploration of cosmological parameters
import numpy as np
import matplotlib.pyplot as plt
from tacc import cosmology, constitutive

def explore_cosmology(kappa_value=2.0, H0_value=70.0, Omega_m_value=0.3):
    """Explore cosmological predictions for different parameters"""
    
    Omega_de_value = 1.0 - Omega_m_value  # Flat universe
    
    print(f"Cosmological Parameters:")
    print(f"  κ = {kappa_value:.2f}")
    print(f"  H₀ = {H0_value:.1f} km/s/Mpc")
    print(f"  Ωₘ = {Omega_m_value:.3f}")
    print(f"  Ωᵈᵉ = {Omega_de_value:.3f}")
    
    # Distance modulus comparison
    z_range = np.linspace(0.01, 2.0, 100)
    mu_tacc = cosmology.distance_modulus_tacc(z_range, kappa_value, H0_value, Omega_m_value, Omega_de_value)
    mu_lcdm = cosmology.distance_modulus_tacc(z_range, 2.0, H0_value, Omega_m_value, Omega_de_value)
    
    plt.figure(figsize=(15, 10))
    
    # Distance modulus
    plt.subplot(2, 3, 1)
    plt.plot(z_range, mu_tacc, 'b-', linewidth=2, label=f'TACC (κ={kappa_value})')
    plt.plot(z_range, mu_lcdm, 'r--', linewidth=2, label='ΛCDM (κ=2.0)')
    plt.xlabel('Redshift z')
    plt.ylabel('Distance Modulus μ (mag)')
    plt.title('Distance-Redshift Relation')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Hubble parameter evolution
    H_z = np.array([cosmology.hubble_parameter_tacc(z, kappa_value, H0_value, Omega_m_value, Omega_de_value) for z in z_range])
    H_z_lcdm = np.array([cosmology.hubble_parameter_tacc(z, 2.0, H0_value, Omega_m_value, Omega_de_value) for z in z_range])
    
    plt.subplot(2, 3, 2)
    plt.plot(z_range, H_z, 'b-', linewidth=2, label=f'TACC (κ={kappa_value})')
    plt.plot(z_range, H_z_lcdm, 'r--', linewidth=2, label='ΛCDM (κ=2.0)')
    plt.xlabel('Redshift z')
    plt.ylabel('H(z) (km/s/Mpc)')
    plt.title('Hubble Parameter Evolution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Constitutive law visualization
    N_range = np.linspace(0.1, 1.2, 100)
    B_values = constitutive.B_of_N(N_range, kappa_value)
    
    plt.subplot(2, 3, 3)
    plt.plot(N_range, B_values, 'g-', linewidth=2)
    plt.axvline(x=1.0, color='k', linestyle=':', alpha=0.7, label='Present (N=1)')
    plt.axhline(y=1.0, color='k', linestyle=':', alpha=0.7)
    plt.xlabel('N (Computational Capacity ~ a(t))')
    plt.ylabel('B(N) = exp[-κ(1-N)]')
    plt.title('Constitutive Law')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Dark energy density evolution
    rho_de_eff = []
    for z in z_range:
        a = 1.0 / (1.0 + z)
        B_factor = constitutive.B_of_N(a, kappa_value)
        rho_de_eff.append(Omega_de_value * B_factor)
    
    plt.subplot(2, 3, 4)
    plt.plot(z_range, rho_de_eff, 'purple', linewidth=2, label=f'TACC (κ={kappa_value})')
    plt.axhline(y=Omega_de_value, color='orange', linestyle='--', linewidth=2, label='ΛCDM')
    plt.xlabel('Redshift z')
    plt.ylabel('Effective Dark Energy Density')
    plt.title('Dark Energy Evolution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Residual from ΛCDM
    residual_mu = mu_tacc - mu_lcdm
    
    plt.subplot(2, 3, 5)
    plt.plot(z_range, residual_mu, 'red', linewidth=2)
    plt.axhline(y=0, color='k', linestyle=':', alpha=0.7)
    plt.xlabel('Redshift z')
    plt.ylabel('Δμ (TACC - ΛCDM) (mag)')
    plt.title('Distance Modulus Deviation')
    plt.grid(True, alpha=0.3)
    
    # Age of universe comparison
    try:
        t_range = np.linspace(1, 13.8, 100)
        a_tacc = cosmology.scale_factor_evolution(t_range, kappa_value, H0_value, Omega_m_value, Omega_de_value)
        a_lcdm = cosmology.scale_factor_evolution(t_range, 2.0, H0_value, Omega_m_value, Omega_de_value)
        
        plt.subplot(2, 3, 6)
        plt.plot(t_range, a_tacc, 'b-', linewidth=2, label=f'TACC (κ={kappa_value})')
        plt.plot(t_range, a_lcdm, 'r--', linewidth=2, label='ΛCDM (κ=2.0)')
        plt.axhline(y=1.0, color='k', linestyle=':', alpha=0.7, label='Present')
        plt.xlabel('Cosmic Time (Gyr)')
        plt.ylabel('Scale Factor a(t)')
        plt.title('Scale Factor Evolution')
        plt.legend()
        plt.grid(True, alpha=0.3)
    except:
        plt.subplot(2, 3, 6)
        plt.text(0.5, 0.5, 'Scale factor\nintegration\nfailed', 
                ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('Scale Factor Evolution')
    
    plt.tight_layout()
    plt.show()
    
    # Print key predictions
    print(f"\nKey Predictions at z=1:")
    z_test = 1.0
    mu_test = cosmology.distance_modulus_tacc(z_test, kappa_value, H0_value, Omega_m_value, Omega_de_value)
    mu_test_lcdm = cosmology.distance_modulus_tacc(z_test, 2.0, H0_value, Omega_m_value, Omega_de_value)
    print(f"  Distance modulus: {mu_test:.3f} mag (ΛCDM: {mu_test_lcdm:.3f})")
    print(f"  Difference: {mu_test - mu_test_lcdm:.3f} mag")
    
    H_test = cosmology.hubble_parameter_tacc(z_test, kappa_value, H0_value, Omega_m_value, Omega_de_value)
    H_test_lcdm = cosmology.hubble_parameter_tacc(z_test, 2.0, H0_value, Omega_m_value, Omega_de_value)
    print(f"  Hubble parameter: {H_test:.1f} km/s/Mpc (ΛCDM: {H_test_lcdm:.1f})")

# Try different parameter combinations
print("Exploring different cosmological parameters:")
for kappa in [0.5, 1.5, 2.0, 2.5, 3.5]:
    explore_cosmology(kappa, 70.0, 0.3)
    print("\n" + "="*70 + "\n")

## Advanced Analysis: Parameter Space Exploration

In [ ]:
# Comprehensive parameter space analysis
def comprehensive_cosmology_analysis():
    """Detailed analysis of TACC cosmological parameter space"""
    
    # Parameter grids
    kappa_range = np.linspace(0.5, 4.0, 50)
    z_range = np.linspace(0.01, 3.0, 100)
    
    # Generate synthetic supernova data for comparison
    z_sn, mu_sn, mu_err_sn = cosmology.generate_synthetic_supernova_data(
        z_max=2.0, n_points=50, kappa_true=2.0, noise_level=0.1
    )
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Chi-squared landscape
    chi2_values = []
    for kappa in kappa_range:
        mu_theory = cosmology.distance_modulus_tacc(z_sn, kappa)
        residuals = (mu_sn - mu_theory) / mu_err_sn
        chi2 = np.sum(residuals**2)
        chi2_values.append(chi2)
    
    chi2_values = np.array(chi2_values)
    chi2_min = np.min(chi2_values)
    
    ax1.plot(kappa_range, chi2_values, 'b-', linewidth=2)
    ax1.axhline(y=chi2_min + 1, color='orange', linestyle=':', label='1σ')
    ax1.axhline(y=chi2_min + 4, color='red', linestyle=':', label='2σ')
    ax1.axvline(x=2.0, color='green', linestyle='--', label='GR (κ=2)')
    ax1.set_xlabel('κ (Constitutive Parameter)')
    ax1.set_ylabel('χ² (Supernova Fit)')
    ax1.set_title('Parameter Constraints from Synthetic Data')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Distance modulus contours
    Z, K = np.meshgrid(z_range[:50], kappa_range[:30])  # Reduce grid for speed
    MU = np.zeros_like(Z)
    
    for i, kappa in enumerate(kappa_range[:30]):
        for j, z in enumerate(z_range[:50]):
            MU[i, j] = cosmology.distance_modulus_tacc(z, kappa)
    
    contours = ax2.contour(Z, K, MU, levels=20, colors='blue', alpha=0.6)
    ax2.clabel(contours, inline=True, fontsize=8)
    ax2.set_xlabel('Redshift z')
    ax2.set_ylabel('κ (Constitutive Parameter)')
    ax2.set_title('Distance Modulus Contours')
    ax2.axhline(y=2.0, color='red', linestyle='--', label='GR (κ=2)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Dark energy equation of state
    # w_eff = -1 + (1/3)(d ln ρ_de / d ln a)
    a_range = np.linspace(0.2, 1.0, 100)
    
    for kappa in [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]:
        w_eff = []
        for a in a_range:
            # Approximate w from B(a) derivative
            B_a = constitutive.B_of_N(a, kappa)
            da = 0.01
            if a + da <= 1.0:
                B_a_plus = constitutive.B_of_N(a + da, kappa)
                dB_da = (B_a_plus - B_a) / da
                w_eff_val = -1 + (1/3) * (a / B_a) * dB_da
            else:
                w_eff_val = -1
            w_eff.append(w_eff_val)
        
        z_eff = 1.0/np.array(a_range) - 1.0
        ax3.plot(z_eff, w_eff, linewidth=2, label=f'κ={kappa}')
    
    ax3.axhline(y=-1, color='black', linestyle='--', alpha=0.7, label='ΛCDM (w=-1)')
    ax3.axhline(y=-1/3, color='gray', linestyle=':', alpha=0.7, label='Radiation (w=-1/3)')
    ax3.set_xlabel('Redshift z')
    ax3.set_ylabel('w_eff (Dark Energy EoS)')
    ax3.set_title('Effective Dark Energy Equation of State')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.set_xlim([0, 4])
    ax3.set_ylim([-2, 0])
    
    # Observable differences from ΛCDM
    observable_diff = []
    z_obs = np.linspace(0.1, 2.0, 50)
    
    for kappa in kappa_range:
        mu_tacc = cosmology.distance_modulus_tacc(z_obs, kappa)
        mu_lcdm = cosmology.distance_modulus_tacc(z_obs, 2.0)
        rms_diff = np.sqrt(np.mean((mu_tacc - mu_lcdm)**2))
        observable_diff.append(rms_diff)
    
    ax4.semilogy(kappa_range, observable_diff, 'purple', linewidth=2)
    ax4.axvline(x=2.0, color='green', linestyle='--', alpha=0.7, label='GR (κ=2)')
    ax4.axhline(y=0.01, color='orange', linestyle=':', alpha=0.7, label='0.01 mag (detectable)')
    ax4.set_xlabel('κ (Constitutive Parameter)')
    ax4.set_ylabel('RMS Distance Modulus Difference (mag)')
    ax4.set_title('Observable Deviations from ΛCDM')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Find best-fit κ
    best_fit_idx = np.argmin(chi2_values)
    kappa_best = kappa_range[best_fit_idx]
    
    print(f"Parameter Space Analysis Results:")
    print(f"  Best-fit κ from synthetic data: {kappa_best:.3f}")
    print(f"  Minimum χ²: {chi2_min:.2f}")
    print(f"  Observable deviations > 0.01 mag for |κ-2| > {np.interp(0.01, observable_diff, np.abs(kappa_range-2.0)):.2f}")
    
    # Constraint summary
    print(f"\nObservational Constraints:")
    print(f"  Current supernovae precision: ~0.15 mag")
    print(f"  Future surveys (LSST): ~0.01 mag")
    print(f"  TACC distinguishable from ΛCDM if |κ-2| > 0.1")

print("Running comprehensive cosmological parameter analysis...")
comprehensive_cosmology_analysis()

## Key Insights

1. **Natural Dark Energy**: TACC provides a computational origin for cosmic acceleration without requiring a cosmological constant

2. **Scale Factor as Capacity**: The universe's computational capacity N ≈ a(t) naturally evolves with cosmic time

3. **Observable Predictions**: κ ≠ 2 produces measurable deviations in distance-redshift relations and Hubble parameter evolution

4. **Parameter Constraints**: Future supernova surveys can constrain κ to percent-level precision

5. **Modified Expansion History**: TACC predicts different early universe behavior compared to ΛCDM

## Physical Interpretation

- **Early Universe**: Small scale factor → limited computational capacity → different expansion dynamics
- **Present Acceleration**: Computational constraints naturally drive accelerated expansion
- **Dark Energy**: Emerges from B(N) modifying effective energy density evolution
- **Future Evolution**: Computational capacity limits may affect far-future cosmology

**Note**: This framework provides an information-theoretic foundation for dark energy, potentially unifying quantum information and cosmology through computational capacity constraints.

## Observational Tests

**Current Constraints:**
- Type Ia supernovae distance moduli
- Cosmic microwave background power spectrum
- Baryon acoustic oscillations

**Future Observations:**
- LSST supernova surveys (2025+)
- Euclid weak lensing (2024+)
- James Webb Space Telescope high-z observations

**Distinguishing Features:**
- Modified distance-redshift relation
- Different dark energy equation of state evolution
- Computational capacity signatures in early universe

## Troubleshooting

**Common Issues:**
- If plots don't display, ensure matplotlib is installed
- Integration failures for extreme κ values are expected and handled
- Memory issues with large parameter grids can be resolved by reducing grid density

**Expected Output:**
- Comprehensive cosmological expansion analysis with 4-panel plots
- Parameter fitting results showing best-fit κ values
- Interactive parameter space exploration
- Observable constraint analysis

**Key Results:**
- κ = 2.0 recovers standard ΛCDM cosmology
- κ ≠ 2.0 predicts observable deviations in supernova data
- Computational capacity provides natural dark energy mechanism
- Future surveys can distinguish TACC from ΛCDM